In [1]:
# Testing Notebook
# Final code in qkd.py

In [16]:
# BB84

# Basis = {"rectilinear", "diagonal"}
# Polarization = {"H, "V", "PD", "ND"}

basis = ["HV", "D"]

polarization = {"1": {"HV": "H", "D": "PD"},
                "0": {"HV": "V", "D": "ND"}


In [ ]:
# qubit

class qubit():
    
    self.__init__(self):
        self.state = (0, 1)

In [14]:
import random

def generate_key(n):
    """
    :param n: length of bitstring 
    """
    bit_value = [0, 1]
    
    return "".join([str(random.choice(bit_value)) for i in range(n)])

def select_basis(n):
    """
    :param n: length of bitstring 
    """
    
    

In [15]:
generate_key(1000)

'111110110101010111110100000110101000101101111011111110011101100110010000101000010111101000110010010111111001011100100001101000011011001101011111011111010101110100111011101101011110010001000101001100100001010000101010100100100011011100011111011010010111111000101010000010100110100100011111100010010000010001100010001110000011010101100100011001010101111001101100100010010010101100100000010111011100010100110101110101011111011111100110000100100010100000000000111110111110101101010010101010000001001000110101011011001101101111110011000010000010011100111011010111101110111001111001111010000011111111100111001011101001001111110111101000010101011010101110111010101111000100111001110010101010100110000001100111011101110011111010010101101110101100000010001011100011110110100110100110010000010111001011000110100001110110101010001010101111100001111001001000000110010110111100111010011011000100110000000011010101100100011110101010000110001000000100000100110110000101100110111001101011001101100100111110110001111